In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle

In [14]:
# load dense softmax model
model = load_model('./Data/relu_softmax_model.h5')

In [15]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [16]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [17]:
# make suare images have shape (28, 28, 1)
x_train = np.reshape(x_train, (60000, 784))
x_test = np.reshape(x_test, (10000, 784))

print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 784)
60000 train samples
10000 test samples


In [18]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
a_val_img = x_test[0]

In [20]:
# Cool
model.predict(np.array([a_val_img]))

array([[2.8971952e-16, 4.6572166e-11, 3.6074706e-11, 3.4015815e-09,
        9.2796529e-17, 1.1560394e-16, 1.5715649e-23, 1.0000000e+00,
        7.9251764e-13, 2.7750113e-11]], dtype=float32)

In [21]:
import tf2onnx
import onnxruntime as rt

In [25]:
spec = (tf.TensorSpec((None, 784), tf.float32, name="input"),)
output_path = './Data/relu_softmax_model.onnx'

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]


2022-01-27 19:18:13.479732: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-27 19:18:13.481089: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.01ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-01-27 19:18:13.518726: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-27 19:18:13.525614: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 18 nodes (-6), 23 edges (-6), time = 3.179ms.
  function_optimizer: function_optimizer did nothing. time = 0.049ms.
  

In [27]:
# Run the ONNX model
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": [a_val_img]})

In [28]:
onnx_pred[0]

array([[2.8971952e-16, 4.6572253e-11, 3.6074772e-11, 3.4015946e-09,
        9.2796886e-17, 1.1560396e-16, 1.5715769e-23, 1.0000000e+00,
        7.9252219e-13, 2.7750165e-11]], dtype=float32)

In [29]:
model.predict(np.array([a_val_img]))

array([[2.8971952e-16, 4.6572166e-11, 3.6074706e-11, 3.4015815e-09,
        9.2796529e-17, 1.1560394e-16, 1.5715649e-23, 1.0000000e+00,
        7.9251764e-13, 2.7750113e-11]], dtype=float32)